In [ ]:
library(tibble)
library(jsonlite)
library(tidyverse)
library(tidyr)
library(reshape2)
library(zeallot)

rm(list=ls())

## Asset pairs

assets_url <- "https://api.kraken.com/0/public/AssetPairs"
assets_raw <- fromJSON(assets_url, flatten=TRUE, simplifyDataFrame = TRUE)
assets_list <- do.call(rbind, assets_raw$result)
a1 <- as.tibble(unlist(assets_list[,1]))
a2 <- as.tibble(rownames(assets_list))
asset_pair <- data.frame(a1,a2)
colnames(asset_pair) <- c("short", "long")
asset_pair

btc <- assets_raw$result$XXBTZUSD
f <- lm(btc$fees[,1]~btc$fees[,2],data=as.tibble(btc$fees))$coefficients
fm <- lm(btc$fees_maker[,1]~btc$fees_maker[,2],data=as.tibble(btc$fees_maker))$coefficients
btcx <- 
  tibble(
    btc$wsname,
    btc$lot, 
    btc$pair_decimals, 
    btc$lot_decimals, 
    btc$lot_multiplier, 
    btc$leverage_buy[1],
    btc$leverage_buy[2],
    btc$leverage_sell[1],
    btc$leverage_sell[2],
    f[1],f[2],
    fm[1],fm[2],
    btc$margin_call,
    btc$margin_stop,
    btc$ordermin)

## ticker, etc. data

install.packages("jsonlite")
install.packages("tidyverse")
install.packages("data.table")
install.packages("bit64")
library(jsonlite)
library(tidyverse)
library(data.table)
library(bit64)


options(timeout=400)

## To handle the Kraken timeout 

CatchupPause <- function(Secs){
  Sys.sleep(Secs) #pause to let connection work
    closeAllConnections()k
    showConnections(all=T)
  gc()
}

# DOTUSD	DOTUSD


## Asset pairs
asset_pst <- "DOTUSD"
asset_pst_long <- "DOTUSD"

dpt <- 4  # depth of order book data we want to extract
depth <- spread <-  data.frame()


i <- 0
ticker_data <- data.frame()

while (i < 20000) {
  
     Sys.sleep(1) # one capture per second
  
  tryCatch({
    tryCatch({
      
      ktime <-
        fromJSON("https://api.kraken.com/0/public/Time", flatten = TRUE)$result$unixtime     # Kraken server time
      
      # ticker
      ticker <- "https://api.kraken.com/0/public/Ticker?pair=" %>%  paste0(asset_pst) %>% fromJSON(flatten = TRUE) 
      
      list_tick <- names( eval(parse(text = paste0("ticker$result$", asset_pst))) )  # names from the list of the different types of data from ticker call
      for(k in list_tick) {
        ticker_temp <- paste0("ticker$result$", asset_pst_long,"$", k)
        tick_tick_temp <- as.numeric(eval(parse(text = ticker_temp)))
        tick_temp <- data.frame(i+1,k,tick_tick_temp)
        ticker_data <- bind_rows(ticker_data, tick_temp)
        
        cat("\n ticker element: ",k," = ", eval(parse(text = ticker_temp))  )     ## if you ever need any of this information
      }
      
      # trades
      
      trade_data <-"https://api.kraken.com/0/public/Trades?pair=" %>% paste0(asset_pst)  %>% fromJSON(flatten = TRUE)
      trade_temp <- paste0("trade_data$result$", asset_pst_long)
      trade_e <-  eval(parse(text = trade_temp)) %>% 
        as_tibble(.name_repair = "unique") 

      trade_last_time <-
        trade_data$result$last %>%  as_tibble(.name_repair = "unique") %>% unlist() 
      
      
      if (!is.null(trade_e) & nrow(trade_e) != 0) {
        trade_e <- trade_e[,1:5] %>% bind_cols(ktime, trade_last_time,i)
        colnames(trade_e) <- c("p", "v", "t", "action", "order", "ktime", "last_trade_time", "capture_period")
        if (i==0) {trade <- trade_e}
        if (i!=0) {trade <- bind_rows(trade_e,trade) %>%  as_tibble(.name_repair = "unique")}
      }
      
      # depth
      
      depth_ab <- "https://api.kraken.com/0/public/Depth?pair=" %>%  paste0(asset_pst) %>% fromJSON(flatten = TRUE) 
      
      depth_str_ask <- paste0("depth_ab$result$", asset_pst_long, "$asks")
      depth_ask <- eval(parse(text = depth_str_ask)) %>% as_tibble(.name_repair = "unique")   # as_tibble() keeps the variables from becoming factors
      depth_ask <- depth_ask %>% as_tibble(.name_repair = "unique") %>% 
        bind_cols("a") %>% as_tibble(.name_repair = "unique") %>% 
        bind_cols(ktime) %>% as_tibble(.name_repair = "unique") %>% 
        bind_cols(i) %>% as_tibble(.name_repair = "unique") 
      colnames(depth_ask) <- c("p", "v", "t", "type", "ktime","capture_period")
      depth_ask <- depth_ask  %>%   
        mutate(p = as.numeric(p),
               v = as.numeric(v),
               t = as.numeric(t)) %>%             
        arrange(p)  %>% 
        slice(1:dpt) %>% as_tibble() 
      
      depth_str_bid <- paste0("depth_ab$result$", asset_pst_long, "$bids")
      depth_bid <-  eval(parse(text = depth_str_bid)) %>% as_tibble(.name_repair = "unique")  
      depth_bid <- depth_bid %>% as_tibble(.name_repair = "unique")  %>% 
        bind_cols("b") %>% as_tibble(.name_repair = "unique") %>%
        bind_cols(ktime) %>% as_tibble(.name_repair = "unique")  %>% 
        bind_cols(i) %>% as_tibble(.name_repair = "unique") 
      colnames(depth_bid) <- c("p", "v", "t", "type", "ktime","capture_period")
      depth_bid <- depth_bid %>% 
        mutate(p = as.numeric(p),
               v = as.numeric(v),
               t = as.numeric(t)) %>% 
        arrange(desc(p)) %>% 
        slice(1:dpt) %>% as_tibble() 
      
      if(nrow(depth_ask) == dpt & nrow(depth_bid) == dpt){
        depth <- bind_rows(depth_ask, depth_bid) %>% bind_rows(depth) %>% as_tibble(.name_repair = "unique") 
      }
      
      
      # counter for while loop
      i <- i + 1
      
      if(i %% 20 == 0)  cat("\ncount = ",i)
      close(url("https://api.kraken.com"))
      
      
    }, interrupt = function(x) {
      message("TIMOUT \n Try Again! \n\n")
      CatchupPause(10)
      
      
      i <<- i + 1    # bump up one while-index value
      
    })
    
  }, interrupt = function(x) {
    message("LONG TIMOUT \n Try Again! \n\n")
    CatchupPause(1000)
    
    i <<- i + 1   # bump up one while-index value
    
  })
  
}


save(ticker_data,depth,trade,spread, file = "kraken_data_DOT.RData")
fwrite(depth, file="depth.csv")
fwrite(trade, file="trade.csv")
fwrite(spread, file="spread.csv")

## consolidate


rm(list=ls())
library(tidyverse)
library(data.table)

load("/home/westland/Desktop/bitcoin_research/kraken_data_11.RData")

colnames(spread) <- c("time","bid","ask")
colnames(ticker_data) <- c("capture_period","identifier","value")
depth$o_t <- ceiling(as.numeric(depth$t))
depth$t_p <- as.POSIXct(depth$o_t, origin="1970-01-01")

depth$o_ktime <- ceiling(as.numeric(depth$t))
depth$ktime_p <- as.POSIXct(depth$o_ktime, origin="1970-01-01")

trade$o_t <- ceiling(as.numeric(trade$t))
trade$t_p <- as.POSIXct(trade$o_t, origin="1970-01-01")

trade$last_t <- ceiling(as.numeric(trade$last_trade_time)/1000000000)
trade$last_t_p <- as.POSIXct(trade$last_t, origin="1970-01-01")

trade$o_ktime <- ceiling(as.numeric(trade$t))
trade$ktime_p <- as.POSIXct(trade$o_ktime, origin="1970-01-01")


cat("\n ",nrow(trade))

trade <- trade %>% 
  filter(between(
    as.POSIXct(ktime_p,origin="1970-01-01"), 
    as.POSIXct(min(as.POSIXct(depth$ktime_p,origin="1970-01-01")),origin="1970-01-01"),
    as.POSIXct(max(as.POSIXct(depth$ktime_p,origin="1970-01-01")),origin="1970-01-01"))) %>% 
  arrange(ktime_p) 

cat("\n ",nrow(trade), "\n  ")


depth <- depth %>% arrange(ktime_p)
old_ktime_p <- depth$ktime_p
t_depth <- data.frame()


depth_a <- depth %>% filter(type=="a")
depth_b <- depth %>% filter(type=="b")
as.character(summary(as.POSIXct(trade$ktime_p,origin="1970-01-01")))
as.character(summary(as.POSIXct(depth_a$ktime_p,origin="1970-01-01")))
as.character(summary(as.POSIXct(depth_b$ktime_p,origin="1970-01-01")))

counter <- unique(trade$ktime)

for(i in counter[1:10000]){  #               nrow(trade)){
  t_depth <- depth %>% 
    filter(as.POSIXct(ktime_p,origin="1970-01-01") <= as.POSIXct(i,origin="1970-01-01")) %>% 
    filter(as.POSIXct(ktime_p,origin="1970-01-01") == max(as.POSIXct(ktime_p,origin="1970-01-01"))) 
  
  t_depth_a <- t_depth %>% 
    filter(type == "a") %>% 
    arrange(p) %>% 
    slice(1:4)
  
  t_depth_b <- t_depth %>% 
    filter(type == "b") %>% 
    arrange(desc(p)) %>% 
    slice(1:4) 
  
  t_depth <- t_depth %>% bind_rows(t_depth_a) %>% bind_rows(t_depth_b) 
}

head(as.character(trade$ktime_p))
head(as.character(t_depth$ktime_p))

library(tidyverse)
library(data.table)
library(zoo)
library(bit64)
library(anytime)

load("/home/westland/Desktop/bitcoin_research/kraken_data_2.RData")

##  I will extract all of the market orders from trade
## then I will look at the latest 4 bids and asks for a particular capture period
## I will look, for trades, at the first capture period that has a first time that is later than the most 
## recent time from the book at a particular capture period.


trade$capture_period <- NULL
depth$capture_period <- NULL

cat("\n depth size = ", nrow(depth), "trade size = ", nrow(trade))
depth <- unique(depth)
trade <- unique(trade) %>% filter(order=="m")
cat("\n unique depth size = ", nrow(depth), "trade size = ", nrow(trade))


depth <- depth %>% 
  mutate(p = as.numeric(p),
         v = as.numeric(v),
         t = as.numeric(t)
  )


trade <- trade %>% 
  mutate(p = as.numeric(p),
         v = as.numeric(v),
         t = ceiling(as.numeric(t))
  )

dt <- as.character(summary(as.POSIXct(depth$t, origin="1970-01-01")))
tt <- as.character(summary(as.POSIXct(trade$t, origin="1970-01-01")))
cat("\n depth times = ", dt)
cat("\n trade times = ", tt)


i_depth <- depth %>% filter(t < max(trade$t))
i_trade <- trade %>% filter(t > min(depth$t))

dt <- as.character(summary(as.POSIXct(i_depth$t, origin="1970-01-01")))
tt <- as.character(summary(as.POSIXct(i_trade$t, origin="1970-01-01")))
cat("\n depth times = ", dt)
cat("\n trade times = ", tt)


i_trade <- trade %>%   
  filter(order == "m") %>% 
  group_by(t) %>% 
  summarize(
    p=mean(p),
    v=sum(v)
  ) %>% 
  ungroup() 

lst_trade <- as.numeric(i_trade$t)  ## clock ticks for looking at the order book
dpt <-  4 # depth for moment conditions

for(i in lst_trade){
  
  i_depth_bid <- depth %>% 
    group_by(t) %>% 
    filter(type=="b",
           t <= i)  %>% 
    arrange(t,desc(p)) %>% 
    slice(1:dpt) %>% 
    ungroup() 
  
  i_depth_ask <- depth %>% 
    group_by(t) %>% 
    filter(type=="a",
           t <= i)   %>% 
    arrange(t,p) %>% #                       
    slice(1:dpt) %>% 
    ungroup() 
  
  i_depth <- bind_rows(i_depth_ask, i_depth_bid) %>% arrange(t)  
  
  
}

trade_time <- i_trade %>% 
  group_by(t) %>% 
  summarize( 
    p = sum(p*v)/sum(v),  # weighted mean
    v = sum(v)
  ) %>% 
  ungroup() 

tot <- t_depth_temp <- data.frame()
tm <- unique(trade_time$t)
for(j in tm){
  t_depth <- i_depth %>% 
    filter(t<=tm) %>% 
    slice(1)
  
  
  t_depth <- bind_rows(t_depth,t_depth_temp)
  
}

t_trade <- trade_time %>% 
  filter(t == tm)
t_tot <- inner_join(t_depth,t_trade, by="t")

tot <- bind_rows(t_tot,tot)



# Select contemporaneous depth and trade times


library(tidyverse)
library(data.table)
library(zoo)
library(bit64)

last_trade_time <- as.character.integer64(trade$last_trade_time)

trade_catchup <- trade_catchup %>% 
  group_by(t) %>% slice(1)

depth <- depth %>% 
  group_by(t) %>% slice(1)

ztrade <- as.zoo(trade_catchup, trade_catchup$t_time)
zdepth <- as.zoo(depth, depth$d_time)

zdt <- merge.zoo(ztrade,zdepth)
zdtc <- coredata(zdt)



# Turning the extraction into your standard set of files

library(data.table)
library(tidyverse)


load("~/Desktop/kraken_data.RData")

depth_db<- depth %>% 
  transmute(
    price = p,
    vol = v, 
    time = t,
    type,
    stime  = ktime
  ) %>% as_tibble()


trade_db <- trade %>% 
  transmute(
    price = as.numeric(as.character(p)),
    vol = as.numeric(as.character(v)), 
    time = as.numeric(as.character(t)),
    type = action,
    order,
    stime  = ktime
  )

library(lubridate)
library(readr)


dpt <-  4 # depth for moment conditions
book_best_bid <- depth_db %>%   
  filter(type=="b")  %>% 
  group_by(stime) %>% 
  arrange(desc(price)) %>% 
  slice(1:dpt) %>% 
  ungroup() %>% 
  mutate(stime = floor(as.numeric(stime)),
         signal = "b" )

book_best_ask <- depth_db %>% 
  filter(type=="a") %>% 
  group_by(stime) %>% 
  arrange(price) %>% 
  slice(1:dpt) %>% 
  mutate(stime = floor(as.numeric(stime)),
         signal = "a" )

book_bind <- bind_rows(
  book_best_bid,
  book_best_ask) %>% 
  select(
    price,
    vol,
    stime,
    signal)

# trades
next_mkt_trade <- trade_db %>%   
  filter(order == "m") %>% 
  group_by(time) %>% 
  slice(1) %>% 
  ungroup() %>% 
  transmute(
    price = as.numeric(price) - as.numeric(lag(price)),
    vol = as.numeric(vol),
    stime = stime,
    signal= ifelse(type == "b","tb","ts")
  )    


book_trade_bind <-  
  bind_rows(book_bind, next_mkt_trade) 

del <- book_trade_bind  %>% 
  filter( signal == "tb" | signal == "ts") %>% 
  mutate(price = as.numeric(price) - lag(as.numeric(price)),
         vol=0,
         signal = replace(signal,  signal == "tb" | signal == "ts", "delta"))

book_trade_bind <-  
  bind_rows(book_trade_bind, del) %>% 
  arrange(stime)

fwrite(book_trade_bind, file="/home/westland/Desktop/bitcoin_research/combined_book_mkt_trades.csv")



glosten <- book_trade_bind


## MARKET SIZE MOMENT

## computing the "Market order size" third moment condition
## lambda = mean of the 'vol' in the next market order after this time stamp

lambda <- glosten %>% 
  filter(signal == "tb" | signal == "ts") %>% 
  transmute(lambda = mean(vol)) %>%   ## mean absolute value of bid (-) and ask (+)
  unique() %>% as.numeric()


signed_mo_size <- glosten %>% 
  filter(signal == "tb" | signal == "ts") %>% 
  mutate(signed_mo_size = ifelse(signal == "tb",-1*vol,vol)) %>% 
  select(signed_mo_size,stime,signal)

fwrite(signed_mo_size,file="/home/westland/Desktop/bitcoin_research/signed_mo_size.csv")


## BREAKEVEN MOMENT

## computing the "break-even" first moment conditions
## # of conditions =  dpt <-  4 # depth for moment conditions


dpt <- 4; j <- 0
clicks <- unique(glosten$stime)
clicks <- clicks[clicks>0] 

be_mmt <- data.frame()

for(tm in clicks){
  
  j <- j+1
  lc <- length(clicks)
  if(j %% 1000 == 0){cat("\n time = ", tm, "count = ", j, " out of: ", lc)}
  
  
  g_bf <-  glosten %>%  filter(stime == tm)
  v_bid  <- glosten %>%    
    filter(signal == "b" & stime == tm) %>% 
    arrange(desc(price)) %>% 
    select(v_bid = vol) %>% 
    cumsum() 
  
  v_ask  <- glosten %>% 
    filter(signal == "a" & stime == tm) %>% 
    arrange(price) %>% 
    select(v_ask = vol) %>% 
    cumsum() 
  
  p_ask  <- glosten %>% 
    filter(signal == "a" & stime == tm) %>% 
    arrange(price) %>% 
    select(p_ask = price) 
  
  p_bid  <- glosten %>% 
    filter(signal == "b" & stime == tm) %>% 
    arrange(desc(price)) %>% 
    select(p_bid = price) 
  
  if(nrow(p_ask) != 0 &
     nrow(p_bid) != 0 &
     nrow(v_ask) != 0 &
     nrow(v_bid) != 0 ) {
    be_mmt <- be_mmt %>% bind_rows(data.frame(p_ask,p_bid,v_ask,v_bid, tm))     
  }
  
}


be_mmt <- be_mmt %>% 
  transmute(v_cum_spread = v_ask + v_bid, 
            p_spread =   p_ask - p_bid, 
            stime = tm)

fwrite(be_mmt,file="/home/westland/Desktop/bitcoin_research/breakeven_moment.csv")



## UPDATE MOMENT CONDITIONS

## computing the "Updating restriction"  moment conditions
## # of conditions =  dpt <-  4 # depth for moment conditions

# Allow for lags and leads

dpt <- 4; # count <- 0 dpt is the depth
glosten_LL  <- glosten %>%   
  filter(signal =="a" | signal == "b") %>% 
  arrange(stime) %>% 
  mutate(lag_price = dplyr::lag(price, n = 2*dpt, default = 0),
         lead_price = dplyr::lead(price, n = 2*dpt, default = 0),
         lag_vol = dplyr::lag(vol, n = 2*dpt, default = 0),
         lead_vol = dplyr::lead(vol, n = 2*dpt, default = 0)
  )


del_p_ask  <- glosten_LL %>% 
  filter(signal == "a") %>%  # & stime == tm) %>% 
  arrange(price) %>% 
  mutate(del_p_ask = price - lag_price)

del_p_bid  <- glosten_LL %>% 
  filter(signal == "b") %>%  # & stime == tm) %>% 
  arrange(desc(price)) %>% 
  mutate(del_p_bid = price - lag_price) 


dpt <- 4; j <- 0          


# I do it this way because 'cumsum' doesn't work inside 'group_by'

ud_mmt <- data.frame()
clicks <- unique(glosten_LL$stime)
clicks <- clicks[clicks>0] 
for(tm in clicks){
  
  j <- j+1
  lc <- length(clicks)
  if(j %% 500 == 0) cat("\n time = ", tm, "count = ", j, " out of: ", lc)
  
  v_bid  <- glosten_LL %>%    
    filter(signal == "b" & stime == tm) %>% 
    arrange(desc(price)) %>% 
    select(del_v_bid = vol) %>% 
    cumsum()
  
  v_bid_lead  <- glosten_LL %>%    
    filter(signal == "b" & stime == tm) %>% 
    arrange(desc(price)) %>% 
    transmute(del_v_bid =  lead_vol) %>% 
    cumsum()
  
  v_ask  <- glosten_LL %>% 
    filter(signal == "a" & stime == tm) %>% 
    arrange(price) %>% 
    select(del_v_ask = vol) %>% 
    cumsum()
  
  v_ask_lead  <- glosten_LL %>% 
    filter(signal == "a" & stime == tm) %>% 
    arrange(price) %>% 
    transmute(del_v_ask =  lead_vol) %>% 
    cumsum()
  
  del_v_ask <- as.numeric(unlist(v_ask_lead - v_ask))  
  del_v_bid <- as.numeric(unlist(v_bid_lead - v_bid))
  
  #  cat("\n del_ask = ", del_v_ask, "del_bid =",  del_v_bid)
  
  stime <- tm
  new_ud_mmt_vol <- data.frame(del_v_ask, del_v_bid, stime)
  ud_mmt <- ud_mmt %>% bind_rows(new_ud_mmt_vol)
  
}

new_ud_mmt_price <- data.frame(del_p_ask, del_p_bid)
ud_mmt <- inner_join(new_ud_mmt_price,ud_mmt,by="stime")    



fwrite(ud_mmt,file="/home/westland/Desktop/bitcoin_research/update_moment.csv")



# Create the data matrix


library(tidyverse)
library(readr)
library(data.table)

glosten = read_csv("/home/westland/Desktop/bitcoin_research/combined_book_mkt_trades.csv")
signed_mo_size = read_csv("/home/westland/Desktop/bitcoin_research/signed_mo_size.csv")
be_mmt = read_csv("/home/westland/Desktop/bitcoin_research/breakeven_moment.csv") 
ud_mmt = read_csv("/home/westland/Desktop/bitcoin_research/update_moment.csv") 
# BE

be1 <- be_mmt %>% group_by(stime) %>% 
  slice(1) %>% transmute(z1a = p_spread, y1a = v_cum_spread) 
be2 <- be_mmt %>% group_by(stime) %>% 
  slice(2) %>% transmute(z1b = p_spread, y1b = v_cum_spread)
be3 <- be_mmt %>% group_by(stime) %>% 
  slice(3) %>% transmute(z1c = p_spread, y1c = v_cum_spread) 
be4 <- be_mmt %>% group_by(stime) %>% 
  slice(4) %>% transmute(z1d = p_spread, y1d = v_cum_spread) 

be <- data.frame(be1,be2,be3,be4) %>%  
  select(z1a,y1a,z1b,y1b,z1c,y1c,z1d,y1d, stime) %>% as.data.frame()


# UD

ud1 <- ud_mmt %>% group_by(stime) %>% 
  slice(1) %>%  transmute(z2a = del_p_ask, y2a = del_v_ask) 
ud2 <- ud_mmt %>% group_by(stime) %>% 
  slice(2) %>%  transmute(z2b = del_p_ask, y2b = del_v_ask) 
ud3 <- ud_mmt %>% group_by(stime) %>% 
  slice(3) %>%  transmute(z2c = del_p_ask, y2c = del_v_ask) 
ud4 <- ud_mmt %>% group_by(stime) %>% 
  slice(4) %>%  transmute(z2d = del_p_ask, y2d = del_v_ask) 
ud5 <- ud_mmt %>% group_by(stime) %>% 
  slice(1) %>%  transmute(z2e = del_p_bid, y2e = del_v_bid) 
ud6 <- ud_mmt %>% group_by(stime) %>% 
  slice(2) %>%  transmute(z2f = del_p_bid, y2f = del_v_bid) 
ud7 <- ud_mmt %>% group_by(stime) %>% 
  slice(3) %>%  transmute(z2g = del_p_bid, y2g = del_v_bid) 
ud8 <- ud_mmt %>% group_by(stime) %>% 
  slice(4) %>%  mutate(z2h = del_p_bid, y2h = del_v_bid) 

ud <- bind_cols(ud1,ud2,ud3,ud4,ud5,ud6,ud7,ud8) %>%  
  select(z2a,y2a,z2b,y2b,z2c,y2c,z2d,y2d,z2e,y2e,z2f,y2f,z2g,y2g,z2h,y2h,stime = stime...1) %>%
  as.data.frame()

## GMM_DATA IS ALL MESSED UP ... ALL OF THE VALUES ARE THE SAME FOR ALL ROWS ... ud and be are OK so it must be this next command that is messed up.

gmm_data <- inner_join(ud, be, by="stime") 

temp_mo_size <-  signed_mo_size %>% select(stime, x = signed_mo_size) 


fwrite(gmm_data, file="/home/westland/Desktop/bitcoin_research/gmm_data.csv")


library(tidyverse)


gmm_data <- inner_join(ud, be, by="stime") 

mo <-  signed_mo_size %>% select(stime, x = signed_mo_size) 

ct <- unique(mo$stime) %>%  unlist() %>% as.integer()

# a_mo <- cut(mo$stime, ct)

dat <- read.table("clipboard", header=TRUE)

cuts <- apply(dat, 2, cut, c(-Inf,seq(0.5, 1, 0.1), Inf), labels=0:6)
cuts[cuts=="6"] <- "0"
cuts <- as.data.frame(cuts)



## Estimation


library(gmm)
library(mfp)

library(momentfit)
library(gmm4)


g <- function(th,x) {
  attach(gmm_data)
  alpha <- th[1]
  gamma <- th[2]
  c <- th[3]
  lambda <- th[4]
  
  m1 <- z1a - 2 * gamma - alpha * y1a + 2 * alpha * lambda
  m2 <- z1b - 2 * gamma - alpha * y1b + 2 * alpha * lambda
  m3 <- z1c - 2 * gamma - alpha * y1c + 2 * alpha * lambda
  m4 <- z1d - 2 * gamma - alpha * y1d + 2 * alpha * lambda
  
  m5 <- z2a - alpha * y2a - c - alpha * x
  m6 <- z2b - alpha * y2b - c - alpha * x
  m7 <- z2c - alpha * y2c - c - alpha * x
  m8 <- z2d - alpha * y2d - c - alpha * x
  
  m9 <- z2e - alpha * y2e - c - alpha * x
  m10 <- z2f - alpha * y2f - c - alpha * x
  m11 <- z2g - alpha * y2g - c - alpha * x
  m12 <- z2h - alpha * y2h - c - alpha * x  
  
  m13 <- abs(x) - lambda
  
  f <- cbind(m1,m2,m3, m4,m5,m6, m7,m8,m9,m10,m11,m12,m13)
  detach(gmm_data)
  return(f)
}

attach(gmm_data)
h=z1a+y1a+z1b+y1b+ z1c+y1c+z1d+ y1d+z2a+ y2a+z2b+ y2b+ z2c+ y2c+ z2d+y2d+z2f+y2f+z2g+y2g+z2h+y2h
attach(gmm_data)

fit <- gmm4(g,h, c(alpha=.7,gamma=.02, c=-.01, lambda=.03), vcov="MDS",  type="onestep")

summary(fit)


a <- depth %>% group_by(stime) %>% summarise(svol = sum(vol))
mean(a$svol)
a <- depth %>% group_by(stime,type) %>% summarise(svol = sum(vol))
ask <- a %>% filter(type=="a")
bid <- a %>% filter(type=="b")
mean(ask$svol)
mean(bid$svol)


library(tidyverse)
library(broom)
library(kableExtra)

print(fit, summary.sysGmm=TRUE) %>%  knitr::kable()


cat("\nEND OF PROGRAM")